# Topics Over Time

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../')) ## needed to import the function.py file
from functions import *
import pandas as pd
import plotly.express as px
import plotly.io as pio

2023-03-12 18:53:46.016871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
## Import docs
filelocation = '../../data/DataTextTrain'
data = pd.read_feather(filelocation)
data = data
docs = data["Kommentar"].values
timestamps = data.yearquarter.to_list()
gender = [data["S_sex"][i] for i in range(len(data["S_sex"]))]

# Load the Original Topic Model
topic_model = BERTopic.load("../../models/BERTTopic_paraphrase-multilingual-MiniLM-L12-v2_merged_oulier_reduced.model")
df_topic = topic_model.get_document_info(docs)


# Create full dataframe with all infomration
df = data.join(df_topic)

# Add grouped Age col
df['S_alter'] = pd.to_numeric(df['S_alter'], errors='coerce')
bins = [0, 24, 44, 64, 100]
labels = ["1-24 Jahre", "25-44 Jahre", "45-64 Jahre", "65-100 Jahre"]
df['S_alter_grouped'] = pd.cut(df['S_alter'], bins=bins, labels=labels)

# Create Dataframe for viz
df_topics_by_quarter = get_topic_ratios(df, timeframe_col='yearquarter', name_col='CustomName', topic_col='Topic')

In [5]:
fig = px.line(df_topics_by_quarter,
              x="yearquarter",
              y="Topic_Ratio",
              color="CustomName",
              hover_data=["yearquarter","CustomName","count_x","Topic_Ratio"],
              color_discrete_sequence=color_discrete_sequence,
              template=template)
fig.update_layout(
    title="Themenrelevanz nach Quartal (2019 - 2022)",
    yaxis_title='Häufigkeit',
    xaxis_title='',
    legend_title="")
fig.update_traces(line_width=2)
#fig.update_yaxes(range=(0,0.2))

pio.write_image(fig,"../../exports/images/Themencluster_Zeitachse_1.svg") #save fig as svg

fig.show()